# GitGuardian Remediation Through APIs




## Authentication

We need to authorize your access to the GitGuardian platform
- Go to your GitGuardian dashboard and click on API, then select the Person Access Tokens menu
- create a new access token and copy it to the clipboard 
- DO NOT paste it into this view.  
- create a new file in your project called "authorization.py" and add the line
    - `token = '<GG API Key>'`
- double check your `.gitignore` file to make sure this stays out of your source control. 
- ALTERNATIVELY - set up and use Hashicorp Vault, or another such crendentials manager. This is likely a better path, but this is an example repo.

In [ ]:
## In this first section we authenticate
import requests
import authorization

auth_headers = {'Authorization' : 'Token {}'.format(authorization.token) }
response = requests.get("https://api.gitguardian.com/v1/health", headers=auth_headers)
print(response)



If we see a <Response [200]> then we are good to go
If you see some other code, check to make sure your Access Token is valid or that your GitGuardian dashboard is avaiable. 

Next up, let's get a list of incidents, filtered with:
- status=TRIGGERED - all incidents that have a status of TRIGGERED 
- date_after=2023-03-01 - incidents that happened after a date YYYY-MM-DDThh:mm:ssZ (hours are optional)
- ordering=-date - in an ordered list shown decending from the most recent event

In [ ]:
import json
response = requests.get("http://api.gitguardian.com/v1/incidents/secrets?date_after=2023-03-01&ordering=-date&status=TRIGGERED", headers=auth_headers)
print(json.dumps(response.json(), indent=2))



## Assigning incidents

Let's assign the most recent incident to someome.  In this case, me.

To do that, we will need to make our first POST command
we need to build a payload that consists of 2 things
1. incident_id - The id of the incident to retrieve
AND
2. email - email of the member to assign. This parameter is mutually exclusive with member_id.
OR
3. member_id - id of the member to assign. This parameter is mutually exclusive with email.

We can copy the `indicent_id` from the JSON output above.

Hopefully you know or can find `email` of the person you want to assign.

How do we find the `member_id`? You can look it up with the API!
For this example, I am filtering it to just lookup `role=owner`.

In [ ]:
response = requests.get("https://api.gitguardian.com/v1/members?role=owner", headers=auth_headers)
print(json.dumps(response.json(), indent=2))


OK, so from our list we now have the 
- `indicent_id` 
- `member_id`, 
- `email`, 
let's build a request to assign the first incident to me. I am going to use member ID since I looked it up.

In [ ]:
my_incident_id = 5936880
my_member_id = 277722
response = requests.post("https://api.gitguardian.com/v1/incidents/secrets/{}/assign".format(my_incident_id), headers=auth_headers, data = {"member_id" : my_member_id})
print(json.dumps(response.json(), indent=2))



You can also unassign incidents, which puts them back to 'triggered' status

Note, in Jupyter notebooks, you can just rerun the block above to reassign it, then skip the next block.

In [ ]:
response = requests.post("https://api.gitguardian.com/v1/incidents/secrets/{}/unassign".format(my_incident_id), headers=auth_headers, data = {"member_id" : my_member_id})
print(json.dumps(response.json(), indent=2))


## Examining an incident

Now that we have been assigned the incident, let's take a look at the incident data. 

In [ ]:
response = requests.get("https://api.gitguardian.com/v1/incidents/secrets/{}".format(my_incident_id), headers=auth_headers)
print(json.dumps(response.json(), indent=2))



## Making a decision
Now I have a decision to make 

1. Do I have enough information to resolve or ignore this incident?
2. Do I need to get more information from the committing author involved?


### Secenario 1: You enough information to resolve or ignore this incident

The next decision is to resolve or to ignore. 

#### Resolving indicents

Let's first resolve it. 
in order to resolve an incident, we need 2 things, 
1. `incident_id` - we already set this above as my_incident_id
2. `secret_revoked` - this is a boolean we pass in as data, either true or false

In [ ]:
response = requests.post("https://api.gitguardian.com/v1/incidents/secrets/{}/resolve".format(my_incident_id), headers=auth_headers, data = {"secret_revoked" : "false"})
print(json.dumps(response.json(), indent=2))



### Reopening incidents

You can also re-open incidents
all we need is the `incident_id`

In [ ]:
response = requests.post("https://api.gitguardian.com/v1/incidents/secrets/{}/reopen".format(my_incident_id), headers=auth_headers)
print(json.dumps(response.json(), indent=2))


### Ignoring Incidents

You can also choose to ingore incidents

Yet again, we need the `indident_id`
But we also need a `ignore_reason`
This has three possible states, expressed as strings
- "test_credential"
- "false_positive"
- "low_risk"

Let's ignore this one as `low_risk`

In [ ]:
response = requests.post("https://api.gitguardian.com/v1/incidents/secrets/{}/ignore".format(my_incident_id), headers=auth_headers, data = {"ignore_reason" : "low_risk"})
print(json.dumps(response.json(), indent=2))

### Scenario 2. I need to get more information from the committing author involved

We can do this by enabling sharing and sharing the generated share URL.

When requesting the share we have some options: 
- auto_healing - Default: false - Allow the developer to resolve or ignore through the share link
- feedback_collection	- Default: true - Allow the developer to submit their feedback through the share link
- lifespan - Default: 0 - Lifespan, in hours, of the share link. If 0 or unset, a default value will be applied based on the workspace settings.

For this example, let's leave `feedback_collection` and `lifespan` alone and set `auto_healing` to `true`


In [ ]:
response = requests.post("https://api.gitguardian.com/v1/incidents/secrets/{}/share".format(my_incident_id), headers=auth_headers, data = {"auto_healing" : "true"})
print(json.dumps(response.json(), indent=2))



## Scanning for secrets with the API

Most of the time when you are scanning a repo or a document, you are going to be better off using ggshield, the CLI.
But in the int

In [ ]:
import os
f = open("authorization.py", "r", encoding="utf-8")
book = f.read()
response = requests.post("https://api.gitguardian.com/v1/scan", headers=auth_headers, data = {"filename":"authorization.py", "document": book})
print(json.dumps(response.json(), indent=2))



## Automate worflows

So far, this kitchen sink demo has served to show
- How the API overallworks
- What JSON responses look like and how to examine them
- The overall workflow of incident assignment, investigation, and resolution with or without developer involvement. 

I hope it has helped your underatanding.

However, everything we have done so far as been manual.
The real power of the API is the ability to automate any workflow we can imagine

Let's automate the same pattern of 
- Checking for unassigned, triggered incidents
- Assigning the most recent incident to a team member
- Let's pre-build the needed link, just in case we need to involve the developer

All we need is the API calls we have already defined and a little bit of JSON parsing with python

In [ ]:
# We imported these earlier, copying here for convienence
# import json
# import requests

# First, let's get our current list of TRIGGERED incidents, most recent listed first.
response = requests.get("http://api.gitguardian.com/v1/incidents/secrets?date_after=2023-03-01&ordering=-date&status=TRIGGERED", headers=auth_headers)

# Let's store this JSON data in a var for ease of coding
incident_data = response.json()

# If there are none, we are done. We can check the number of items with len()
if len(incident_data) <= 0:
    print("There are no more incidents to assign.")
    exit(keep_kernel=True)
else:
    # Assuming there is one, let's get the first incident's id 
    first_incident_id = incident_data[0]['id']
#     print(first_incident_id) 

# Next let's grab the first listed member's id so we can assign them this indicent
response = requests.get("https://api.gitguardian.com/v1/members?role=owner", headers=auth_headers)
assignee_member_id = response.json()[0]['id']
# print(assignee_member_id)

# Now, let's assign the incident to our member
response = requests.post("https://api.gitguardian.com/v1/incidents/secrets/{}/assign".format(first_incident_id), headers=auth_headers, data = {"member_id" : assignee_member_id})
# print(json.dumps(response.json(), indent=2))

# And next, let's get the sharing URL and print it to the terminal
# It might already exist, for our incident, so let's check that first before we send another API call
# When making the request, let's assume we want the dev to be able to resolve or ignore the incident
# Let's print the URL to the terminal

if incident_data[0]['share_url']:
#     print("You already shared it")
    print(incident_data[0]['share_url'])
else:
    response = requests.post("https://api.gitguardian.com/v1/incidents/secrets/{}/share".format(first_incident_id), headers=auth_headers, data = {"auto_healing" : "true"})
    print(response.json()['share_url'])                                                                                                                                               


# And finally, let's show ALL the now updated details about our incident
response = requests.get("https://api.gitguardian.com/v1/incidents/secrets/{}".format(first_incident_id), headers=auth_headers)
print(json.dumps(response.json(), indent=4))
    

In [ ]:
### Unassign function for the above for testing reasons. 
response = requests.post("https://api.gitguardian.com/v1/incidents/secrets/{}/unassign".format(first_incident_id), headers=auth_headers, data = {"member_id" : assignee_member_id})
print(json.dumps(response.json(), indent=2))
